# POS Tagging with Hidden Markov Model

https://www.mygreatlearning.com/blog/pos-tagging/

In [1]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

#download the treebank corpus from nltk
nltk.download('treebank')
 
#download the universal tagset from nltk
nltk.download('universal_tagset')
 
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))
 
#print the first two sentences along with tags
print(nltk_data[:2])

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]]


In [2]:
#print each word with its respective tag for first two sentences
for sent in nltk_data[:2]:
  for tuple in sent:
    print(tuple)

('Pierre', 'NOUN')
('Vinken', 'NOUN')
(',', '.')
('61', 'NUM')
('years', 'NOUN')
('old', 'ADJ')
(',', '.')
('will', 'VERB')
('join', 'VERB')
('the', 'DET')
('board', 'NOUN')
('as', 'ADP')
('a', 'DET')
('nonexecutive', 'ADJ')
('director', 'NOUN')
('Nov.', 'NOUN')
('29', 'NUM')
('.', '.')
('Mr.', 'NOUN')
('Vinken', 'NOUN')
('is', 'VERB')
('chairman', 'NOUN')
('of', 'ADP')
('Elsevier', 'NOUN')
('N.V.', 'NOUN')
(',', '.')
('the', 'DET')
('Dutch', 'NOUN')
('publishing', 'VERB')
('group', 'NOUN')
('.', '.')


In [3]:
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(nltk_data,
                                     train_size=0.80,
                                     test_size=0.20,
                                     random_state = 101)

In [4]:
# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

80310
20366


In [5]:
# check some of the tagged words.
train_tagged_words[:5]

[('Drink', 'NOUN'),
 ('Carrier', 'NOUN'),
 ('Competes', 'VERB'),
 ('With', 'ADP'),
 ('Cartons', 'NOUN')]

In [6]:
#use set datatype to check how many unique tags are present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)
 
# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

12
{'CONJ', 'ADP', '.', 'X', 'DET', 'NUM', 'NOUN', 'VERB', 'ADV', 'ADJ', 'PRON', 'PRT'}


### Emission Probability

The probability of a word categorized as a specific tag is the emission probability.

In [7]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
 
     
    return (count_w_given_tag, count_tag)

### Transition Probability

The probability of a particular sequence of tags which should be high for a grammatically correct sentence.

In [8]:
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [9]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
 
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
 
# convert the matrix to a df for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

,CONJ,ADP,.,X,DET,NUM,NOUN,VERB,ADV,ADJ,PRON,PRT
CONJ,0.000549,0.055982,0.035126,0.009330,0.123491,0.040615,0.349067,0.150384,0.057080,0.113611,0.060373,0.004391
ADP,0.001012,0.016958,0.038724,0.034548,0.320931,0.063275,0.323589,0.008479,0.014553,0.107062,0.069603,0.001266
.,0.060079,0.092908,0.092372,0.025641,0.172192,0.078210,0.218539,0.089690,0.052569,0.046132,0.068769,0.002789
X,0.010379,0.142226,0.160869,0.075726,0.056890,0.003075,0.061695,0.206419,0.025754,0.017682,0.054200,0.185086
DET,0.000431,0.009918,0.017393,0.045134,0.006037,0.022855,0.635906,0.040247,0.012074,0.206411,0.003306,0.000287
NUM,0.014281,0.037487,0.119243,0.202428,0.003570,0.184220,0.351660,0.020707,0.003570,0.035345,0.001428,0.026062
NOUN,0.042454,0.176827,0.240094,0.028825,0.013106,0.009144,0.262344,0.149134,0.016895,0.012584,0.004659,0.043935
VERB,0.005433,0.092357,0.034807,0.215930,0.133610,0.022836,0.110589,0.167956,0.083886,0.066390,0.035543,0.030663
ADV,0.006982,0.119472,0.139255,0.022886,0.071373,0.029868,0.032196,0.339022,0.081458,0.130721,0.012025,0.014740
ADJ,0.016893,0.080583,0.066019,0.020971,0.005243,0.021748,0.696893,0.011456,0.005243,0.063301,0.000194,0.011456


### Viterbi Algorithm

In [10]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [11]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset
random.seed(1234)      #define a random seed to get same sentences when run multiple times
 
# choose random 10 numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]
 
# list of 10 sents on which we test the model
test_run = [test_set[i] for i in rndom]
 
# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]
 
# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]


In [12]:
#Here We will only test 10 sentences to check the accuracy
#as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  29.037946701049805
Viterbi Algorithm Accuracy:  93.77990430622009


In [13]:
#To improve the performance,we specify a rule base tagger for unknown words 
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]
 
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [14]:
#modified Viterbi to include rule based tagger in it
def Viterbi_rule_based(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
     
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                 
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
             
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
        
         
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
             
         
        state.append(state_max)
    return list(zip(words, state))

In [15]:
#test accuracy on subset of test data 
start = time.time()
tagged_seq = Viterbi_rule_based(test_tagged_words)
end = time.time()
difference = end-start
 
print("Time taken in seconds: ", difference)
 
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
 
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  27.420118808746338
Viterbi Algorithm Accuracy:  97.1291866028708


In [16]:
#Check how a sentence is tagged by the two POS taggers
#and compare them
test_sent="Will can see Marry"
pred_tags_rule=Viterbi_rule_based(test_sent.split())
pred_tags_withoutRules= Viterbi(test_sent.split())
print("With Rules:", pred_tags_rule)
print("Without Rules:", pred_tags_withoutRules)
#Will and Marry are tagged as NUM as they are unknown words for Viterbi Algorithm

With Rules: [('Will', 'NOUN'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'NOUN')]
Without Rules: [('Will', 'CONJ'), ('can', 'VERB'), ('see', 'VERB'), ('Marry', 'CONJ')]


Testing a new observation:

The goal of high level competitive swimming is to break personal or world records while beating competitors in any given event.

src: https://en.wikipedia.org/wiki/Swimming_(sport)

In [17]:
test_sent = 'The goal of high level competitive swimming is to break personal or world records while beating competitors in any given event.'
print(test_sent)

The goal of high level competitive swimming is to break personal or world records while beating competitors in any given event.


In [18]:
pred_tags_rule=Viterbi_rule_based(test_sent.split())
pred_tags_withoutRules= Viterbi(test_sent.split())
print("With Rules:", pred_tags_rule,'\n')
print("Without Rules:", pred_tags_withoutRules)

With Rules: [('The', 'DET'), ('goal', 'NOUN'), ('of', 'ADP'), ('high', 'ADJ'), ('level', 'NOUN'), ('competitive', 'ADJ'), ('swimming', 'VERB'), ('is', 'VERB'), ('to', 'PRT'), ('break', 'VERB'), ('personal', 'ADJ'), ('or', 'CONJ'), ('world', 'NOUN'), ('records', 'NOUN'), ('while', 'ADP'), ('beating', 'VERB'), ('competitors', 'NOUN'), ('in', 'ADP'), ('any', 'DET'), ('given', 'VERB'), ('event.', 'NOUN')] 

Without Rules: [('The', 'DET'), ('goal', 'NOUN'), ('of', 'ADP'), ('high', 'ADJ'), ('level', 'NOUN'), ('competitive', 'ADJ'), ('swimming', 'CONJ'), ('is', 'VERB'), ('to', 'PRT'), ('break', 'VERB'), ('personal', 'ADJ'), ('or', 'CONJ'), ('world', 'NOUN'), ('records', 'NOUN'), ('while', 'ADP'), ('beating', 'CONJ'), ('competitors', 'NOUN'), ('in', 'ADP'), ('any', 'DET'), ('given', 'VERB'), ('event.', 'CONJ')]
